In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup as bs
import re

In [2]:
#기존의 크롤링 방법 말고 다른 방식을 써보자

id='mugudeli601'
url = "https://www.law.go.kr/DRF/lawSearch.do?OC={}&target=prec&type=XML".format(id)
response = urlopen(url).read()
bs(response)

c:\Users\Heeyo\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<?xml version="1.0" encoding="UTF-8"?><html><body><precsearch><target>prec</target>&lt;키워드&gt;*키워드&gt;<section>evtNm</section><totalcnt>82497</totalcnt><page>1</page><prec id="1">&lt;판례일련번호&gt;223337판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[부당이득금반환]]&gt;사건명&gt;&lt;사건번호&gt;2018다43128사건번호&gt;&lt;선고일자&gt;2022.06.30선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;판결판결유형&gt;&lt;선고&gt;선고선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=223337&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="2">&lt;판례일련번호&gt;223365판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[법인세경정청구거부처분취소]]&gt;사건명&gt;&lt;사건번호&gt;2018두54323사건번호&gt;&lt;선고일자&gt;2022.06.30선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;세무사건종류명&gt;&lt;사건종류코드&gt;400108사건종류코드&gt;&lt;판결유형&gt;판결판결유형&gt;&lt;선고&gt;선고선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=223365&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="3">&

In [3]:
xtree = ET.fromstring(response) #response에 저장된 string으로부터 tree를 생성한다

In [4]:
#first check what is in it
'''for child in xtree:
    print(':',child.tag,child.attrib)
print(xtree.find('page').text)'''

"for child in xtree:\n    print(':',child.tag,child.attrib)\nprint(xtree.find('page').text)"

In [5]:
xtree[3].text #totalCnt

'82497'

In [6]:
for node in xtree[5:]:
    print('\n\n',node.attrib)
    tags=[]
    for child in node:
        tags.append(child.tag)
    for t in tags:
        print(node.find(t).text)



 {'id': '1'}
223337
부당이득금반환
2018다43128
2022.06.30
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223337&type=HTML&mobileYn=


 {'id': '2'}
223365
법인세경정청구거부처분취소
2018두54323
2022.06.30
대법원
None
세무
400108
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223365&type=HTML&mobileYn=


 {'id': '3'}
223369
횡령
2017도21286
2022.06.30
대법원
None
형사
400102
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223369&type=HTML&mobileYn=


 {'id': '4'}
223345
확약서무효확인의소·손해배상(기)
2019다246696
2022.06.30
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223345&type=HTML&mobileYn=


 {'id': '5'}
223321
공유물분할·부당이득금
2020다210686
2022.06.30
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223321&type=HTML&mobileYn=


 {'id': '6'}
223311
부당전직구제재심판정취소청구의소[원고 회사가 육아휴직을 사용한 참가인을 복직시키면서 기존의 ‘발탁매니저’가 아닌 ‘영업담당’으로 인사발령한 것이 남녀고용평등과 일·가정 양립지원에 관한 법률 제19조 제4항의 ‘휴직 전과 같은 업무 또는 같은 수준의 임금을 지급하는 직무’에 복귀시킬 의무 위반에 해당하는지 여부가 문제된 사건]
2017두76005
202

In [7]:
totalCnt = int(xtree.find('totalCnt').text)#판례 페이지에 totalCnt라는 요소가 존재하는 것 같다. 
#왜 20개밖에 없나 했는데 원래 한 페이지당 20개라고 한다. 
#페이지마다 loop을 돌아가야 하는 것

In [8]:
#판례 목록 만들기 
cnt=0
page = 1
rows = []
for i in trange(int(totalCnt / 20)): #기다리는 바를 만드는 tqdm의 range 함수인 trange
    if i>5:
        break
    try:
        items = xtree[5:]
    except:
        break
        
    for node in items:
        data={}
        tags=[]
        for child in node:
            tags.append(child.tag)
        for t in tags:
            data[t]=re.sub(',','$',str(node.find(t).text))

        rows.append(data)
    cnt += 1
    url = "https://www.law.go.kr/DRF/lawSearch.do?OC=mugudeli601&target=prec&type=XML&page={}".format(page)
    response = urlopen(url).read()
    xtree = ET.fromstring(response)
    page+=1

  0%|          | 6/4124 [00:01<12:36,  5.45it/s]


In [9]:
#rows에 모든 정보가 담겨있음
prec_list=pd.DataFrame(rows,columns=rows[0].keys())
prec_list.head(5)

,판례일련번호,사건명,사건번호,선고일자,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,선고,판례상세링크
0,223337,부당이득금반환,2018다43128,2022.06.30,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
1,223365,법인세경정청구거부처분취소,2018두54323,2022.06.30,대법원,None,세무,400108,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
2,223369,횡령,2017도21286,2022.06.30,대법원,None,형사,400102,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
3,223345,확약서무효확인의소·손해배상(기),2019다246696,2022.06.30,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
4,223321,공유물분할·부당이득금,2020다210686,2022.06.30,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...


In [10]:
prec_list.columns

Index(['판례일련번호', '사건명', '사건번호', '선고일자', '법원명', '법원종류코드', '사건종류명', '사건종류코드',
       '판결유형', '선고', '판례상세링크'],
      dtype='object')

In [11]:
#prec_list를 traverse하며 모든 판례정보를 긁기
import re

contents = ['사건명', '사건번호', '선고일자','판결유형','판시사항', '판결요지', '참조조문', '참조판례', '판례내용']
case_list=pd.DataFrame(columns=contents)

def remove_tag(content):
    cleaned=re.sub(',','$',re.sub('<.*?>', '', content))
    return cleaned

for pid,l in zip(prec_list['판례일련번호'],prec_list['판례상세링크']):
    url="https://www.law.go.kr"+l.replace('HTML','XML')
    response=urlopen(url).read()
    xtree=ET.fromstring(response)
    row={}
    row['판례일련번호']=pid
    for content in contents:
        
        try:
            text=xtree.find(content).text
            text=remove_tag(text)
        except:
            text="-"
        row[content]=text
    case_list=pd.concat([case_list,pd.DataFrame([row],columns=row.keys())],axis=0)
case_list.reset_index(inplace=True)
        
        

        

In [12]:
case_list.head(5)

,index,사건명,사건번호,선고일자,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,판례일련번호
0,0,부당이득금반환,2018다43128,20220630,판결,甲 등이 신축한 집합건물에 관한 구분소유가 성립하기 전에 대지에 관하여 근저당권...,甲 등이 신축한 집합건물에 관한 구분소유가 성립하기 전에 대지에 관하여 근저당권...,집합건물의 소유 및 관리에 관한 법률 제2조$ 제20조$ 민사집행법 제91조 제2항,-,【원고$ 피상고인】 원고【피고$ 상고인】 피고【원심판결】 서울북부지법 2018. 9...,223337
1,0,법인세경정청구거부처분취소,2018두54323,20220630,판결,[1] 甲 주식회사가 乙 주식회사를 흡수합병하였는데 乙 회사가 보유하던 甲 회사...,[1] 甲 주식회사가 乙 주식회사를 흡수합병하였는데 乙 회사가 보유하던 甲 회사...,[1] 구 법인세법(2015. 12. 15. 법률 제13555호로 개정되기 전의 ...,-,【원고$ 상고인】 쿠쿠홀딩스주식회사 (소송대리인 법무법인(유한) 광장 담당변호사 마...,223365
2,0,횡령,2017도21286,20220630,판결,횡령죄에서 말하는 ‘보관’의 의미 / 횡령죄의 성립에 필요한 위탁관계는 횡령죄로...,형법 제355조 제1항이 정한 횡령죄에서 보관이란 위탁관계에 따라 재물을 점유하는 ...,형법 제355조 제1항,대법원 2016. 5. 19. 선고 2014도6992 전원합의체 판결(공2016상$...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 법무법인 하늘 담당변호사 이기...,223369
3,0,확약서무효확인의소·손해배상(기),2019다246696$ 246702,20220630,판결,甲 주식회사의 근로자인 乙 등이 희망퇴직을 신청하면서 작성하여 제출한 확약서에 ...,甲 주식회사의 근로자인 乙 등이 희망퇴직을 신청하면서 작성하여 제출한 확약서에 ...,약관의 규제에 관한 법률 제2조 제1호$ 제6조$ 제8조$ 제30조$ 근로기준법 제...,-,【원고(반소피고)$ 피상고인】 원고(반소피고) 1 외 1인 (소송대리인 법무법인(유...,223345
4,0,공유물분할·부당이득금,2020다210686$ 210693,20220630,판결,[1] 공유물분할청구의 소가 고유필수적 공동소송인지 여부(적극) 및 소송계속 중...,[1] 공유물분할청구소송은 분할을 청구하는 공유자가 원고가 되어 다른 공유자 전...,[1] 민법 제269조$ 민사소송법 제51조$ 제67조$ 제81조$ 제82조 / ...,[1] 대법원 2014. 1. 29. 선고 2013다78556 판결(공2014상$...,【원고(반소피고)$ 피상고인】 원고(반소피고) (소송대리인 변호사 이상열)【피고(반...,223321


In [13]:
from pathlib import Path  
filepath = Path('case_list.csv')  
case_list.to_csv(filepath,index=False)

In [14]:
case_list.head(10)

,index,사건명,사건번호,선고일자,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,판례일련번호
0,0,부당이득금반환,2018다43128,20220630,판결,甲 등이 신축한 집합건물에 관한 구분소유가 성립하기 전에 대지에 관하여 근저당권...,甲 등이 신축한 집합건물에 관한 구분소유가 성립하기 전에 대지에 관하여 근저당권...,집합건물의 소유 및 관리에 관한 법률 제2조$ 제20조$ 민사집행법 제91조 제2항,-,【원고$ 피상고인】 원고【피고$ 상고인】 피고【원심판결】 서울북부지법 2018. 9...,223337
1,0,법인세경정청구거부처분취소,2018두54323,20220630,판결,[1] 甲 주식회사가 乙 주식회사를 흡수합병하였는데 乙 회사가 보유하던 甲 회사...,[1] 甲 주식회사가 乙 주식회사를 흡수합병하였는데 乙 회사가 보유하던 甲 회사...,[1] 구 법인세법(2015. 12. 15. 법률 제13555호로 개정되기 전의 ...,-,【원고$ 상고인】 쿠쿠홀딩스주식회사 (소송대리인 법무법인(유한) 광장 담당변호사 마...,223365
2,0,횡령,2017도21286,20220630,판결,횡령죄에서 말하는 ‘보관’의 의미 / 횡령죄의 성립에 필요한 위탁관계는 횡령죄로...,형법 제355조 제1항이 정한 횡령죄에서 보관이란 위탁관계에 따라 재물을 점유하는 ...,형법 제355조 제1항,대법원 2016. 5. 19. 선고 2014도6992 전원합의체 판결(공2016상$...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 법무법인 하늘 담당변호사 이기...,223369
3,0,확약서무효확인의소·손해배상(기),2019다246696$ 246702,20220630,판결,甲 주식회사의 근로자인 乙 등이 희망퇴직을 신청하면서 작성하여 제출한 확약서에 ...,甲 주식회사의 근로자인 乙 등이 희망퇴직을 신청하면서 작성하여 제출한 확약서에 ...,약관의 규제에 관한 법률 제2조 제1호$ 제6조$ 제8조$ 제30조$ 근로기준법 제...,-,【원고(반소피고)$ 피상고인】 원고(반소피고) 1 외 1인 (소송대리인 법무법인(유...,223345
4,0,공유물분할·부당이득금,2020다210686$ 210693,20220630,판결,[1] 공유물분할청구의 소가 고유필수적 공동소송인지 여부(적극) 및 소송계속 중...,[1] 공유물분할청구소송은 분할을 청구하는 공유자가 원고가 되어 다른 공유자 전...,[1] 민법 제269조$ 민사소송법 제51조$ 제67조$ 제81조$ 제82조 / ...,[1] 대법원 2014. 1. 29. 선고 2013다78556 판결(공2014상$...,【원고(반소피고)$ 피상고인】 원고(반소피고) (소송대리인 변호사 이상열)【피고(반...,223321
5,0,부당전직구제재심판정취소청구의소[원고 회사가 육아휴직을 사용한 참가인을 복직시키면서 ...,2017두76005,20220630,판결,남녀고용평등과 일·가정 양립 지원에 관한 법률 제19조 제3항의 ‘불리한 처우’의 ...,남녀고용평등과 일·가정 양립 지원에 관한 법률(이하 ‘남녀고용평등법’이라 한다) 제...,남녀고용평등과 일·가정 양립 지원에 관한 법률 제4조$ 제5조$ 제19조 제1항$ ...,-,【원고$ 피상고인】 롯데쇼핑 주식회사 (소송대리인 법무법인(유한) 광장 담당변호사 ...,223311
6,0,공제금등청구의소,2022다212594,20220630,판결,[1] 공인중개사가 자기가 조사·확인하여 설명할 의무는 없으나 중개의뢰인이 계약...,[1] 공인중개사는 자기가 조사·확인하여 설명할 의무가 없는 사항이라도 중개의뢰...,[1] 공인중개사법 제25조$ 제29조 제1항$ 제30조$ 공인중개사법 시행령 제...,[1] 대법원 1999. 5. 14. 선고 98다30667 판결(공1999상$ 1...,【원고$ 상고인】 원고 1 외 1인 (소송대리인 변호사 한성영)【피고$ 피상고인】 ...,223315
7,0,의사면허취소처분취소[의료법위반죄로 금고 이상의 형의 집행유예를 선고받은 자가 유예기...,2021두62171,20220630,판결,면허취소사유를 정한 구 의료법 제65조 제1항 단서 제1호의 ‘제8조 각호의 어...,구 의료법(2016. 5. 29. 법률 제14220호로 개정되기 전의 것$ 이하 ‘...,구 의료법(2016. 5. 29. 법률 제14220호로 개정되기 전의 것) 제8조 ...,대법원 1998. 2. 13. 선고 97누18042 판결(공1998상$ 789)$ ...,【원고$ 상고인】 원고 (소송대리인 법무법인(유한) 원 담당변호사 이주웅 외 3인)...,223305
8,0,특정경제범죄가중처벌등에관한법률위반(사기)·자본시장과금융투자업에관한법률위반·근로기준법...,2021도244,20220630,판결,변호인 또는 변호인이 되려는 자의 접견교통권 행사의 한계 및 접견교통권이 한계를...,변호인 또는 변호인이 되려는 자의 접견교통권은 신체구속제도 본래의 목적을 침해하...,헌법 제12조 제4항$ 형법 제137조$ 형사소송법 제34조,대법원 2007. 1. 31. 자 2006모657 결정$ 대법원 2009. 4. 2...,【피 고 인】 피고인【상 고 인】 피고인 및 검사【변 호 인】 법무법인(유한) 케이...,223351
9,0,업무상배임[역설계 등의 방법으로 입수 가능한 상태에 있는 회사의 정보를 무단으로 반...,2018도4794,20220630,판결,회사 직원이 경쟁업체 또는 스스로의 이익을 위하여 이용할 의사로 무단으로 자료를...,회사 직원이 경쟁업체 또는 스스로의 이익을 위하여 이용할 의사로 무단으로 자료를...,형법 제355조 제2항$ 제356조,대법원 2011. 6. 30. 선고 2009도3915 판결(공2011하$ 1549)...,【피 고 인】 피고인 1 외 2인【상 고 인】 피고인들【변 호 인】 법무법인 혜명 ...,223361


In [15]:
print(case_list['참조판례'])

0                                                      -
1                                                      -
2      대법원 2016. 5. 19. 선고 2014도6992 전원합의체 판결(공2016상$...
3                                                      -
4       [1] 대법원 2014. 1. 29. 선고 2013다78556 판결(공2014상$...
                             ...                        
115    대법원 2003. 5. 30. 선고 2003다13512 판결(공2003하$ 1434...
116     [1] 대법원 1995. 11. 10. 선고 93누16468 판결(공1995하$ ...
117     대법원 2007. 12. 13. 선고 2007다57459 판결(공2008상$ 23)\n
118                                                    -
119     [1][2] 대법원 2021. 11. 11. 선고 2017다222368 판결(공2...
Name: 참조판례, Length: 120, dtype: object


In [16]:
type(case_list)

pandas.core.frame.DataFrame

In [17]:
nonan_assistive=case_list[~(case_list['참조판례']=='-')].copy()
nonan_assistive[nonan_assistive['참조판례'].str.contains('\[[0-9]\]')]

,index,사건명,사건번호,선고일자,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,판례일련번호
4,0,공유물분할·부당이득금,2020다210686$ 210693,20220630,판결,[1] 공유물분할청구의 소가 고유필수적 공동소송인지 여부(적극) 및 소송계속 중...,[1] 공유물분할청구소송은 분할을 청구하는 공유자가 원고가 되어 다른 공유자 전...,[1] 민법 제269조$ 민사소송법 제51조$ 제67조$ 제81조$ 제82조 / ...,[1] 대법원 2014. 1. 29. 선고 2013다78556 판결(공2014상$...,【원고(반소피고)$ 피상고인】 원고(반소피고) (소송대리인 변호사 이상열)【피고(반...,223321
6,0,공제금등청구의소,2022다212594,20220630,판결,[1] 공인중개사가 자기가 조사·확인하여 설명할 의무는 없으나 중개의뢰인이 계약...,[1] 공인중개사는 자기가 조사·확인하여 설명할 의무가 없는 사항이라도 중개의뢰...,[1] 공인중개사법 제25조$ 제29조 제1항$ 제30조$ 공인중개사법 시행령 제...,[1] 대법원 1999. 5. 14. 선고 98다30667 판결(공1999상$ 1...,【원고$ 상고인】 원고 1 외 1인 (소송대리인 변호사 한성영)【피고$ 피상고인】 ...,223315
11,0,적격심사대상자지위확인,2022다209383,20220630,판결,[1] 계약담당 공무원이 입찰절차에서 지방자치단체를 당사자로 하는 계약에 관한 ...,[1] 계약담당 공무원이 입찰절차에서 지방자치단체를 당사자로 하는 계약에 관한 ...,[1] 민법 제103조$ 지방자치단체를 당사자로 하는 계약에 관한 법률 제1조$ ...,[1] 대법원 2001. 12. 11. 선고 2001다33604 판결(공2002상...,【원고$ 피상고인】 유한회사 신성산업개발 (소송대리인 법무법인 이우스 담당변호사 이...,223317
13,0,손해배상(기),2018다276218,20220630,판결,[1] 부동산처분금지가처분등기의 효력 및 처분행위가 가처분에 저촉되는 것인지 판...,[1] 부동산에 관하여 처분금지가처분의 등기가 된 후에 가처분채권자가 본안소송에...,[1] 민사집행법 제300조 제1항$ 제305조 제3항$ 부동산등기법 제4조 / ...,[1] 대법원 2003. 2. 28. 선고 2000다65802$ 65819 판결(...,【원고$ 상고인】 주식회사 오스카부동산리츠 (소송대리인 변호사 이종근)【피고$ 피상...,223343
14,0,상속재산분할·상속재산분할·상속재산분할·상속재산분할,2017스98$ 99$ 100$ 101,20220630,결정,[1] 상속재산분할의 기준이 되는 구체적 상속분을 산정하는 방법 [2] 상속개...,[1] 상속재산분할은 법정상속분이 아니라 특별수익(피상속인의 공동상속인에 대한 ...,[1] 민법 제269조$ 제1008조$ 제1008조의2$ 제1009조$ 제1013...,[1] 대법원 1995. 3. 10. 선고 94다16571 판결(공1995상$ ...,【청구인(반심판상대방)$ 피재항고인】 청구인(반심판상대방) 1 외 4인 (소송대리인...,223313
18,0,양수금[타인의 은행계좌로 송금한 금액이 착오송금임을 주장하며 그 수취은행을 상대로 ...,2016다237974,20220630,판결,[1] 예금거래기본약관에 따라 송금의뢰인이 수취인의 예금계좌에 자금이체를 하여 ...,[1] 자금이체는$ 은행 간 및 은행점포 간의 송금절차를 통하여 저렴한 비용으로...,[1] 민법 제702조 / [2] 민법 제492조$ 제702조$ 제741조,[1] 대법원 2007. 11. 29. 선고 2007다51239 판결(공2007하...,【원고$ 상고인】 주식회사 비엔씨 (소송대리인 법무법인 에이디엘 담당변호사 양창길 ...,223357
19,0,등록무효(상),2022후10128,20220630,판결,[1] 구 상표법 제6조 제1항 제3호의 규정 취지 및 어떤 상표가 위 규정에서...,[1] 구 상표법(2016. 2. 29. 법률 제14033호로 전부 개정되기 전...,[1] 구 상표법(2016. 2. 29. 법률 제14033호로 전부 개정되기 전의...,[1] 대법원 1991. 3. 27. 선고 90후1208 판결(공1991$ 129...,【원고$ 피상고인】 주식회사 피플앤코 (소송대리인 변리사 박수조)【피고$ 상고인】 ...,223367
24,0,공유물분할·부당이득금,2020다210686$ 210693,20220630,판결,[1] 공유물분할청구의 소가 고유필수적 공동소송인지 여부(적극) 및 소송계속 중...,[1] 공유물분할청구소송은 분할을 청구하는 공유자가 원고가 되어 다른 공유자 전...,[1] 민법 제269조$ 민사소송법 제51조$ 제67조$ 제81조$ 제82조 / ...,[1] 대법원 2014. 1. 29. 선고 2013다78556 판결(공2014상$...,【원고(반소피고)$ 피상고인】 원고(반소피고) (소송대리인 변호사 이상열)【피고(반...,223321
26,0,공제금등청구의소,2022다212594,20220630,판결,[1] 공인중개사가 자기가 조사·확인하여 설명할 의무는 없으나 중개의뢰인이 계약...,[1] 공인중개사는 자기가 조사·확인하여 설명할 의무가 없는 사항이라도 중개의뢰...,[1] 공인중개사법 제25조$ 제29조 제1항$ 제30조$ 공인중개사법 시행령 제...,[1] 대법원 1999. 5. 14. 선고 98다30667 판결(공1999상$ 1...,【원고$ 상고인】 원고 1 외 1인 (소송대리인 변호사 한성영)【피고$ 피상고인】 ...,223315
31,0,적격심사대상자지위확인,2022다209383,20220630,판결,[1] 계약담당 공무원이 입찰절차에서 지방자치단체를 당사자로 하는 계약에 관한 ...,[1] 계약담당 공무원이 입찰절차에서 지방자치단체를 당사자로 하는 계약에 관한 ...,[1] 민법 제103조$ 지방자치단체를 당사자로 하는 계약에 관한 법률 제1조$ ...,[1] 대법원 2001. 12. 11. 선고 2001다33604 판결(공2002상...,【원고$ 피상고인】 유한회사 신성산업개발 (소송대리인 법무법인 이우스 담당변호사 이...,223317
